In [4]:
from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI

llm = OpenAI(model_name="gpt-3.5-turbo-instruct")
chat = ChatOpenAI(model_name="gpt-3.5-turbo")

a = llm.predict("How many planets are there in the solar system?")
b = chat.predict("How many planets are there in the solar system?")

a, b

('\n\nThere are eight planets in the solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.',
 'There are eight planets in the solar system.')